# Module 9: Understanding Transformers & LLMs

**Goal:** Understand how modern language models work and when to use them

**Time:** ~25 minutes

**What you'll do:**
1. Understand attention (the key innovation)
2. Use a pre-trained transformer for sentiment
3. Compare prompting vs fine-tuning vs RAG
4. Evaluate LLM outputs for hallucinations

---

## Setup

In [ ]:
# Install transformers if needed
try:
    from transformers import pipeline
except ImportError:
    !pip install transformers torch -q
    from transformers import pipeline

import pandas as pd
import numpy as np

# Load support tickets
tickets = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/streamcart_tickets.csv')

print(f"Loaded {len(tickets):,} support tickets")

---

## Part 1: What Makes Transformers Special?

The key innovation: **Attention** lets the model weigh which words matter for each word.

In [ ]:
attention_explanation = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  THE ATTENTION INTUITION                                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Old models (RNN/LSTM): Read text left-to-right, one word at a time.       │
│  Like reading with a flashlight - can only see nearby context.             │
│                                                                             │
│  Transformers (Attention): See ALL words at once, decide which matter.     │
│  Like reading with the lights on - every word can connect to every other.  │
│                                                                             │
│  Example: "The cat sat on the mat because it was tired"                    │
│  - What does "it" refer to?                                                │
│  - Attention learns: "it" attends strongly to "cat" (not "mat")           │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

WHY THIS MATTERS:
  ✓ Captures long-range dependencies
  ✓ Parallelizable (much faster training)
  ✓ Scales to billions of parameters
"""
print(attention_explanation)

---

## Part 2: Using Pre-trained Transformers

HuggingFace `pipeline` makes it easy to use transformers.

In [ ]:
# Load a sentiment analysis model (DistilBERT - smaller, faster)
print("Loading sentiment model (this may take a minute)...")
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
print("✓ Model loaded!")

In [ ]:
# Analyze some tickets
sample_tickets = [
    "I love your service! Best subscription ever!",
    "This is terrible. I want my money back immediately.",
    "Can I change my shipping address?",
    "WHY WAS I CHARGED TWICE???",
    "Just wanted to say thanks for the great products!"
]

print("=== Sentiment Analysis ===")
for ticket in sample_tickets:
    result = sentiment_model(ticket)[0]
    emoji = "😊" if result['label'] == 'POSITIVE' else "😠"
    print(f"{emoji} [{result['label']}] ({result['score']:.0%}) {ticket}")

In [ ]:
# TODO: Analyze sentiment of real tickets
#
# Take 20 random tickets and analyze their sentiment

sample = tickets.sample(20, random_state=42)

results = []
for idx, row in sample.iterrows():
    try:
        sentiment = sentiment_model(row['ticket_text'][:512])[0]  # Truncate long text
        results.append({
            'ticket_text': row['ticket_text'],
            'category': row['category'],
            'sentiment': sentiment['label'],
            'confidence': sentiment['score']
        })
    except:
        pass

results_df = pd.DataFrame(results)

# Summary
print("=== Sentiment by Category ===")
print(results_df.groupby('category')['sentiment'].value_counts())

---

## Part 3: The LLM Decision Framework

When should you use prompting, RAG, or fine-tuning?

In [ ]:
decision_framework = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  APPROACH        │  WHEN TO USE                    │  EXAMPLE               │
├─────────────────────────────────────────────────────────────────────────────┤
│  PROMPTING       │  General knowledge tasks        │  "Summarize this email"│
│  (zero/few-shot) │  No custom data needed          │  "Write product copy"  │
│                  │  Quick prototyping              │  "Extract key points"  │
├─────────────────────────────────────────────────────────────────────────────┤
│  RAG             │  Need current/company data      │  "What's our policy?" │
│  (retrieval +    │  Knowledge changes often        │  Answer from docs      │
│   generation)    │  Accuracy is critical           │  Customer support bot  │
├─────────────────────────────────────────────────────────────────────────────┤
│  FINE-TUNING     │  Specific format/style needed   │  Legal document review │
│                  │  Domain-specific language       │  Medical transcription │
│                  │  Classification tasks           │  Ticket routing        │
└─────────────────────────────────────────────────────────────────────────────┘

RULE OF THUMB:
  1. Try prompting first (cheapest, fastest)
  2. Add RAG if you need specific knowledge
  3. Fine-tune only if quality still insufficient
"""
print(decision_framework)

In [ ]:
# TODO: For each scenario, decide: Prompting, RAG, or Fine-tuning?

scenarios = [
    "Auto-generate email responses to common questions",
    "Answer customer questions about our return policy",  
    "Classify support tickets into categories",
    "Summarize long customer feedback into bullet points",
    "Chat with customers using company knowledge base"
]

# Your answers:
answers = [
    "???",  # Auto-generate email responses
    "???",  # Answer about return policy
    "???",  # Classify tickets
    "???",  # Summarize feedback
    "???"   # Chat with knowledge base
]

for s, a in zip(scenarios, answers):
    print(f"• {s}")
    print(f"  → {a}\n")

---

## Part 4: Hallucination Detection

LLMs can confidently generate false information. How do you catch it?

In [ ]:
hallucination_types = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  HALLUCINATION TYPE    │  EXAMPLE                       │  HOW TO DETECT    │
├─────────────────────────────────────────────────────────────────────────────┤
│  Factual invention     │  "Our CEO founded the company  │  Source checking  │
│                        │   in 1985" (wrong date)        │  Knowledge lookup │
├─────────────────────────────────────────────────────────────────────────────┤
│  Source fabrication    │  "According to a 2023 study"   │  Citation verify  │
│                        │   (study doesn't exist)        │  Ask for links    │
├─────────────────────────────────────────────────────────────────────────────┤
│  Confident speculation │  "The product will be avail-   │  Check confidence │
│                        │   able on June 1st" (guessing) │  vs uncertainty   │
├─────────────────────────────────────────────────────────────────────────────┤
│  Context drift         │  Starts correct, drifts to     │  Fact-check each  │
│                        │   unrelated/incorrect content  │  claim separately │
└─────────────────────────────────────────────────────────────────────────────┘

MITIGATION STRATEGIES:
  1. Use RAG to ground responses in real data
  2. Ask model to quote sources
  3. Add "I don't know" as a valid response
  4. Human review for high-stakes content
"""
print(hallucination_types)

---

## Part 5: Evaluating LLM Quality

Traditional metrics don't work well. What to use instead?

In [ ]:
evaluation_methods = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  METHOD               │  WHAT IT MEASURES               │  WHEN TO USE      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Human evaluation     │  Quality, coherence, accuracy   │  Gold standard    │
│  (1-5 ratings)        │  Appropriateness of response    │  but expensive    │
├─────────────────────────────────────────────────────────────────────────────┤
│  LLM-as-Judge         │  Another LLM rates the output   │  Scalable proxy   │
│  (GPT-4 evals)        │  Cheaper than humans            │  for human eval   │
├─────────────────────────────────────────────────────────────────────────────┤
│  Faithfulness         │  Does output match source?      │  RAG applications │
│  (NLI models)         │  Entailment checking            │  Fact-checking    │
├─────────────────────────────────────────────────────────────────────────────┤
│  Task-specific        │  Accuracy, F1, exact match      │  Classification,  │
│  metrics              │  BLEU, ROUGE for generation     │  extraction tasks │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(evaluation_methods)

In [ ]:
# Simple evaluation rubric for support responses

rubric = """
=== Customer Support Response Rubric ===

RELEVANCE (1-5)
  1: Completely off-topic
  3: Partially addresses the question
  5: Directly answers what was asked

ACCURACY (1-5)
  1: Contains false information
  3: Mostly correct, minor issues
  5: Factually correct, matches policies

HELPFULNESS (1-5)
  1: Customer would still be confused
  3: Provides some guidance
  5: Clear next steps, issue resolved

TONE (1-5)
  1: Inappropriate/rude
  3: Neutral
  5: Professional, empathetic
"""
print(rubric)

---

## Part 6: Quick RAG Prototype

Demonstrate the concept of retrieval-augmented generation.

In [ ]:
# Simulated knowledge base
knowledge_base = [
    "Our return policy allows returns within 30 days of purchase.",
    "Shipping typically takes 5-7 business days for standard delivery.",
    "Premium members get free shipping on all orders.",
    "To cancel your subscription, go to Settings > Subscription > Cancel.",
    "Refunds are processed within 5-10 business days."
]

# Create simple retriever
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

kb_vectorizer = TfidfVectorizer()
kb_vectors = kb_vectorizer.fit_transform(knowledge_base)

def retrieve(query, top_k=2):
    """Find most relevant knowledge base entries."""
    query_vec = kb_vectorizer.transform([query])
    sims = cosine_similarity(query_vec, kb_vectors)[0]
    top_idx = sims.argsort()[::-1][:top_k]
    return [knowledge_base[i] for i in top_idx]

In [ ]:
# Demonstrate retrieval
test_queries = [
    "How do I get my money back?",
    "How long until my package arrives?",
    "I want to cancel"
]

print("=== RAG Retrieval Demo ===")
for query in test_queries:
    print(f"\nQuery: '{query}'")
    print("Retrieved context:")
    for doc in retrieve(query):
        print(f"  • {doc}")

In [ ]:
# The RAG prompt template
rag_template = """
=== RAG Prompt Template ===

Use the following context to answer the customer question.
If the answer isn't in the context, say "I don't have that information."

Context:
{retrieved_documents}

Customer Question: {question}

Answer:
"""

print(rag_template)

---

## 📝 Final Exercise: Explain It

Your VP asks: "Should we use ChatGPT to respond to all customer support tickets automatically?"

Write a 5-6 sentence response covering the risks and recommended approach.

In [ ]:
# Write your response:

vp_response = """
YOUR RESPONSE HERE
"""

print(vp_response)

---

## ✅ Module 9 Complete!

**What you learned:**
- How attention makes transformers powerful
- How to use pre-trained models with HuggingFace
- When to use prompting vs RAG vs fine-tuning
- How to detect and mitigate hallucinations

In [ ]:
# Summary
print("=== Module 9 Summary ===")
print("\nKey decision points:")
print("  • Prompting: General tasks, quick prototypes")
print("  • RAG: Need company/current data")
print("  • Fine-tuning: Specific format, domain language")
print("\nHallucination mitigation: RAG + source citations + human review")

**Next:** [Module 10: Building with AI Agents →](./module_10_agents.ipynb)